In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

from datetime import datetime, timedelta 

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import glob
import os

import seaborn
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA

import tensorflow as tf

import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.activations import relu, tanh, linear
from keras.layers import Dropout

from keras.callbacks import EarlyStopping, TerminateOnNaN, ModelCheckpoint
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor

from joblib import dump, load

from sklearn.utils import shuffle
from sklearn.metrics import make_scorer

import json

In [ ]:
trainPath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonTrain\\Versuche08-12_09_2023_train.csv"
valPath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\LSTMjsonVal\\Versuche08-12_09_2023_val.csv"
pipePath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\pipelineMLP.p"
filepath = "C:\\Users\\wch002\\Desktop\\RotorTempDRZ\\8Tempsensors\\cvModel.h5"

In [ ]:
df_train = pd.read_csv(trainPath)
df_val = pd.read_csv(valPath)

y_train = df_train["No.6"]
parameters = ["T_KLEMMUNG", "T_LAGER", "DRZ5", "T_MOTOR", "I_FELD", "V_QUER"]
x_train = df_train[parameters].to_numpy()

y_val = df_val["No.6"]
x_val = df_val[parameters].to_numpy()

In [ ]:
seq_steps = [("standardize_values", StandardScaler()), ("pca", PCA(6)), ("standard_values", StandardScaler())]
pipeline = Pipeline(steps=seq_steps)
dump(pipeline, pipePath)

In [ ]:
newpipeline = load(pipePath)
x_transform = newpipeline.fit_transform(x_train)
x_test = newpipeline.transform(x_val)
newpipeline.steps[1][1].explained_variance_ratio_

In [ ]:
earlyStop = EarlyStopping(monitor='val_loss', patience=100, verbose =1, mode = "auto")
stopNaN = TerminateOnNaN()

In [ ]:
def get_mlp_model(layers=[5, 5, 5], dropout=0, activation = "relu", optimizer="Nadam", first_activ = "relu"):
    model = Sequential()
    first = True
    for layer in layers:
        if first:
            model.add(Dense(layer, activation=first_activ, input_dim=6))
            first = False
        else:
            model.add(Dense(layer, activation=activation))
        model.add(Dropout(dropout))

    model.add(Dense(1, activation="linear"))
    # compile the model
    model.compile(
        optimizer=optimizer,
        loss=ks.losses.mean_absolute_error)
    # return compiled model
    return model

In [ ]:
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
model = KerasRegressor(build_fn=get_mlp_model, verbose=1, layers=[5, 5, 5], dropout=0, activation = "relu", optimizer="Nadam", first_activ = "relu")     
                    #, first_rec_activ="sigmoid", rec_activ="sigmoid"
#model.fit(x_transform, y_train)

In [ ]:
grid = dict( 
    layers = [[5], [10, 5], [500, 500, 300], [100, 150, 50], 
            [700, 500, 300], [700, 700, 500]],    #[500, 400, 200], [700, 700, 500, 300], [700, 700, 500, 100],[25, 25, 25, 20, 20, 20, 10, 10],  [75, 75, 50, 50, 50, 40, 40, 20]
    dropout = [0, 0.1, 0.3, 0.2],
    first_activ = ["relu", "tanh"],
    activation = ["relu", "tanh"],
    #first_rec_activ = ["sigmoid", "tanh"],
    #rec_activ = ["sigmoid", "tanh"],
    optimizer = ["Nadam", "Adam"]
    #losses = [tf.keras.losses.mean_squared_error, tf.keras.losses.mean_absolute_error]
)

searcher = RandomizedSearchCV(estimator=model, n_jobs=1, #cv=[[x_transform, y_train], [x_test, y_val]],
    param_distributions=grid, scoring='neg_mean_absolute_error', n_iter = 5)
searchResults = searcher.fit(x_transform, y_train)

In [ ]:
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_
print("[INFO] best score is {:.2f} using {}".format(bestScore, bestParams))

bestModel = searchResults.best_estimator_
bestModel.fit(x_transform, y_train)
score = bestModel.score(x_val, y_val)
print("score: {:.2f}%".format(score * 100))

In [ ]:
best_pred = bestModel.predict(x_val)
y_pred_train = bestModel.predict(x_train)
error_avg = np.mean(abs(y_val - best_pred))
error_max = max(abs(y_val - best_pred))
print("max: ", error_max)
print("avg: ", error_avg)

In [ ]:
diff_test = y_val - best_pred
error_avg_test = np.mean(abs(diff_test))
error_max_test = np.max(abs(diff_test))

diff_train = y_train - y_pred_train
error_avg_train = np.mean(abs(diff_train))
error_max_train = np.max(abs(diff_train))

#mean_absolute = tf.keras.losses.mean_absolute_error(y_cv[:,0], y_pred.flatten())
#print("mean absolute: ", mean_absolute)
print("Validation scores: ")
print("max error: ", error_max_test)
print("mean abs error: ", error_avg_test)
print("Error over the training data: ")
print("max error: ", error_max_train)
print("mean abs error: ", error_avg_train)